In [1]:
import matplotlib.pyplot as plt

In [2]:
from pkg.planner.moveit.moveit_py import MoveitCompactPlanner_BP, ObjectType, ObjectMPC

In [3]:
from pkg.utils.utils import *
gtimer = GlobalTimer.instance()

In [4]:
# urdf_path, srdf_path = './moveit_plan_compact/test_assets/custom_robots.urdf', './moveit_plan_compact/test_assets/custom_robots.srdf'
urdf_path, srdf_path, config_path = './robots/custom_robots.urdf', './robots/custom_robots.srdf', './robots/'

In [5]:
planner = MoveitCompactPlanner_BP(urdf_path, srdf_path, ["indy0", "panda1"], './robots/')

In [10]:

gtimer.tic("plan_py")
trajectory, success = planner.plan_py("indy0", "indy0_tcp", (-0.3,-0.2,0.4,0,0,0,1), "base_link",
                                      (0, 0, -1.57, 0, -1.57, 0, 0, -0.4, 0, -1.57, 0, 1.57, 1.57),"RRTConnectkConfigDefault",0.1)
gtimer.toc("plan_py")
print(gtimer)
print(success)

plan_py: 	230.0 ms/5 = 46.06 ms (32.187/70.899)

True


In [7]:
gtimer.tic("set_scene")
planner.set_scene([
    ObjectMPC("box", ObjectType.BOX, "base_link", (-0.3,-0.2,0.4,0,0,0,1), (0.1,0.1,0.1), ["base_link", "indy0_link0"], True)
])
gtimer.toc("set_scene")

0.15807151794433594

In [21]:

gtimer.tic("plan_py")
trajectory, success = planner.plan_py("indy0", "indy0_tcp", (-0.3,-0.2,0.4,0,0,0,1), "base_link",
                                      (0, 0, -1.57, 0, -1.57, 0, 0, -0.4, 0, -1.57, 0, 1.57, 1.57), 0.1)
gtimer.toc("plan_py")
print(gtimer)
print(success)

robot/tool/link/goal/Q0: 
 indy0/indy0_tcp/base_link/(-0.3, -0.2, 0.4, 0, 0, 0, 1)/(0, 0, -1.57, 0, -1.57, 0, 0, -0.4, 0, -1.57, 0, 1.57, 1.57) - True
plan_py: 	778.0 ms/9 = 86.444 ms (30.439/123.894)
set_scene: 	1.0 ms/5 = 0.188 ms (0.162/0.232)

True


In [12]:
planner.remove_object(ObjectMPC("box", ObjectType.BOX, "base_link", attach=True))

True

In [13]:
gtimer.tic("plan_py")
trajectory, success = planner.plan_py("indy0", "indy0_tcp", (-0.3,-0.2,0.4,0,0,0,1), "base_link",
                                      (0, 0, -1.57, 0, -1.57, 0, 0, -0.4, 0, -1.57, 0, 1.57, 1.57), 0.1)
gtimer.toc("plan_py")
print(gtimer)
print(success)

robot/tool/link/goal/Q0: 
 indy0/indy0_tcp/base_link/(-0.3, -0.2, 0.4, 0, 0, 0, 1)/(0, 0, -1.57, 0, -1.57, 0, 0, -0.4, 0, -1.57, 0, 1.57, 1.57) - False
plan_py: 	438.0 ms/5 = 87.569 ms (30.439/123.894)
set_scene: 	0.0 ms/2 = 0.191 ms (0.187/0.195)

False


In [14]:
planner.remove_object(ObjectMPC("box", ObjectType.BOX, "base_link", attach=False))

True

In [15]:
gtimer.tic("plan_py")
trajectory, success = planner.plan_py("indy0", "indy0_tcp", (-0.3,-0.2,0.4,0,0,0,1), "base_link",
                                      (0, 0, -1.57, 0, -1.57, 0, 0, -0.4, 0, -1.57, 0, 1.57, 1.57), 0.1)
gtimer.toc("plan_py")
print(gtimer)
print(success)

robot/tool/link/goal/Q0: 
 indy0/indy0_tcp/base_link/(-0.3, -0.2, 0.4, 0, 0, 0, 1)/(0, 0, -1.57, 0, -1.57, 0, 0, -0.4, 0, -1.57, 0, 1.57, 1.57) - True
plan_py: 	469.0 ms/6 = 78.223 ms (30.439/123.894)
set_scene: 	0.0 ms/2 = 0.191 ms (0.187/0.195)

True


In [11]:
planner.add_object(
    ObjectMPC("box", ObjectType.BOX, "base_link", (-0.3,-0.2,0.4,0,0,0,1), (0.1,0.1,0.), ["base_link", "indy0_link0"]))

True

In [12]:
gtimer.tic("plan_py")
trajectory, success = planner.plan_py("indy0", "indy0_tcp", (-0.3,-0.2,0.4,0,0,0,1), "base_link",
                                      (0, 0, -1.57, 0, -1.57, 0, 0, -0.4, 0, -1.57, 0, 1.57, 1.57), 0.1)
gtimer.toc("plan_py")
print(gtimer)
print(success)

set_scene: 	0.0 ms/1 = 0.162 ms (0.162/0.162)
plan_py: 	392.0 ms/4 = 97.951 ms (31.505/122.612)

False


In [13]:
planner.clear_scene()

In [14]:
gtimer.tic("plan_py")
trajectory, success = planner.plan_py("indy0", "indy0_tcp", (-0.3,-0.2,0.4,0,0,0,1), "base_link",
                                      (0, 0, -1.57, 0, -1.57, 0, 0, -0.4, 0, -1.57, 0, 1.57, 1.57), 0.1)
gtimer.toc("plan_py")
print(gtimer)
print(success)

set_scene: 	0.0 ms/1 = 0.162 ms (0.162/0.162)
plan_py: 	418.0 ms/5 = 83.682 ms (26.603/122.612)

True


In [16]:

from scipy.spatial.transform import Rotation

In [18]:
ans = Rotation.from_dcm(np.identity(3)).as_quat()

In [19]:
(1,2)+(3,4)

(1, 2, 3, 4)